In [1]:
from pyspark.sql import SparkSession

spark = (
    SparkSession.builder
    .appName("Delta-Fix")
    .master("spark://spark-master:7077")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.2.0")
    # ship JARs local (đường dẫn trong container Jupyter) tới executors
    # Delta config
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.jars", "/opt/spark/jars/delta-spark_2.12-3.2.0.jar")
    # MinIO
    .config("spark.hadoop.fs.s3a.endpoint", "http://minio:9000")
    .config("spark.hadoop.fs.s3a.path.style.access", "true")
    .config("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
    .getOrCreate()
)


In [2]:
spark

In [3]:
conf = spark._jsc.hadoopConfiguration()
print(conf.get("fs.s3a.path.style.access"))
print("S3 Endpoint:", conf.get("fs.s3a.endpoint"))
print("Access Key:", conf.get("fs.s3a.access.key"))
print("Secret Key:", conf.get("fs.s3a.secret.key"))

true
S3 Endpoint: http://minio:9000
Access Key: minio_access_key
Secret Key: minio_secret_key


In [4]:
src_csv = "s3a://datalake/warehouse/data.csv"
dst_parquet = "s3a://datalake/warehouse/parquet_file/orders"

In [ ]:
df = spark.read.option("header", True).csv(src_csv)

In [16]:


df2 = df.dropna(subset=["CustomerID"]).dropDuplicates(["CustomerID"])
df2.write.mode("overwrite").parquet(dst_parquet)
spark.stop

NameError: name 'df' is not defined

In [17]:
spark.conf.get("spark.sql.catalog.spark_catalog")


'org.apache.spark.sql.delta.catalog.DeltaCatalog'